In [4]:
# test upload
!curl -X POST "http://localhost:8000/upload" \
     -H "accept: application/json" \
     -H "Content-Type: multipart/form-data" \
     -F "file=@snowflake.png"

{"success":true,"image_url":"/static/uploads/snowflake.png","local_path":"static/uploads/snowflake.png"}

In [5]:
# analyze ratio
!curl -X POST "http://localhost:8000/analyze" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/uploads/snowflake.png"

{"success":true,"ratio_string":"1.0:1.0:0.1","data":{"width":1.0,"length":1.0,"thickness":0.1,"complexity":"moderate"},"error":null}

In [11]:
# Generate Silhouette
!curl -X POST "http://localhost:8000/generate-silhouette" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/uploads/snowflake.png"

{"success":true,"image_url":"/static/processed/snowflake_silhouette.png","local_path":"static/processed/snowflake_silhouette.png"}

In [12]:
!curl -X POST "http://localhost:8000/edit-silhouette" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/uploads/snowflake_toedit.png"

{"detail":"Error editing silhouette: Missing required arguments; Expected either ('image' and 'prompt') or ('image', 'prompt' and 'stream') arguments to be given"}

In [5]:
# Convert to 3D model
!curl -X POST "http://localhost:8000/convert-to-3d" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/processed/snowflake_toedit_updated.png" \
     -d "depth_div_width=0.2"

{"success":true,"model_url":"/static/models/snowflake_toedit_updated_3d.stl","local_path":"static/models/snowflake_toedit_updated_3d.stl"}

## Model Refinement Testing

Test the new refinement features for adjusting existing models.

In [13]:
# Refine existing depth map only (returns refined image)
!curl -X POST "http://localhost:8000/refine-model" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "depth_map_path=static/processed/snowflake_toedit_updated.png" \
     -d "refinement_instructions=Make it 20% bigger and add subtle texture to the surface"

{"success":true,"image_url":"/static/processed/snowflake_toedit_updated_refined.png","local_path":"static/processed/snowflake_toedit_updated_refined.png"}

In [14]:
# Refine and regenerate 3D model in one step
!curl -X POST "http://localhost:8000/refine-and-regenerate" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "depth_map_path=static/processed/snowflake_toedit_updated.png" \
     -d "refinement_instructions=Increase depth by 30% and smooth the edges" \
     -d "depth_div_width=0.25" \
     -d "aspect_ratio=1.0"

{"success":true,"refined_image_url":"/static/processed/snowflake_toedit_updated_refined.png","refined_image_path":"static/processed/snowflake_toedit_updated_refined.png","new_model_url":"/static/models/snowflake_toedit_updated_refined_3d.stl","new_model_path":"static/models/snowflake_toedit_updated_refined_3d.stl"}

## OBJ Model Generation with Colors

Test the new OBJ format with color support.

In [15]:
# Generate OBJ with uniform light gray color
!curl -X POST "http://localhost:8000/convert-to-obj" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/processed/snowflake_silhouette.png" \
     -d "depth_div_width=0.2" \
     -d "aspect_ratio=1.0" \
     -d "base_color_r=0.8" \
     -d "base_color_g=0.8" \
     -d "base_color_b=0.8" \
     -d "use_depth_coloring=false"

{"success":true,"obj_url":"/static/models/snowflake_silhouette_3d.obj","obj_path":"static/models/snowflake_silhouette_3d.obj","mtl_url":"/static/models/snowflake_silhouette_3d.mtl","mtl_path":"static/models/snowflake_silhouette_3d.mtl"}

In [16]:
# Generate OBJ with height-based color gradient (Blue->Green->Red)
!curl -X POST "http://localhost:8000/convert-to-obj" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/processed/snowflake_silhouette.png" \
     -d "depth_div_width=0.2" \
     -d "use_depth_coloring=true" \
     -d "color_map=height"

{"success":true,"obj_url":"/static/models/snowflake_silhouette_3d.obj","obj_path":"static/models/snowflake_silhouette_3d.obj","mtl_url":"/static/models/snowflake_silhouette_3d.mtl","mtl_path":"static/models/snowflake_silhouette_3d.mtl"}

In [17]:
# Generate OBJ with regional colors (Red center, Blue edges)
!curl -X POST "http://localhost:8000/convert-to-obj-regional" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/processed/snowflake_silhouette.png" \
     -d "depth_div_width=0.2" \
     -d 'color_config=[{"area":"center","color":[1.0,0.0,0.0]},{"area":"edges","color":[0.0,0.0,1.0]}]'

{"success":true,"obj_url":"/static/models/snowflake_silhouette_colored_3d.obj","obj_path":"static/models/snowflake_silhouette_colored_3d.obj","mtl_url":"/static/models/snowflake_silhouette_colored_3d.mtl","mtl_path":"static/models/snowflake_silhouette_colored_3d.mtl"}

In [1]:
# Generate OBJ with multiple regional colors
!curl -X POST "http://localhost:8000/convert-to-obj-regional" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/processed/snowflake_silhouette.png" \
     -d "depth_div_width=0.2" \
     -d 'color_config=[{"area":"center","color":[1.0,0.5,0.0]},{"area":"top","color":[0.0,1.0,0.0]},{"area":"bottom","color":[1.0,1.0,0.0]},{"area":"edges","color":[0.5,0.0,1.0]}]'

{"success":true,"obj_url":"/static/models/snowflake_silhouette_colored_3d.obj","obj_path":"static/models/snowflake_silhouette_colored_3d.obj","mtl_url":"/static/models/snowflake_silhouette_colored_3d.mtl","mtl_path":"static/models/snowflake_silhouette_colored_3d.mtl"}

## Natural Language Color Adjustment

Test the new feature that allows users to adjust colors using casual descriptions like "make it redder", "add pink tone", etc.

In [18]:
# Adjust single color: "make it redder"
!curl -X POST "http://localhost:8000/adjust-colors" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "user_instruction=make it redder" \
     -d 'current_config={"mode":"single","base_color":[0.8,0.8,0.8]}'

{"success":true,"updated_config":{"mode":"single","base_color":[0.9,0.7,0.7],"explanation":"Increased red channel to 0.9, reduced green and blue to 0.7 to make the color redder"},"explanation":"Increased red channel to 0.9, reduced green and blue to 0.7 to make the color redder","original_instruction":"make it redder"}

## 完整的颜色调整工作流程

**两步流程说明：**

1. **第1步：调整颜色** - `/adjust-colors` 返回新的颜色配置（JSON）
2. **第2步：生成OBJ** - `/convert-to-obj` 使用新配置生成实际的3D模型文件

**注意**: `/adjust-colors` 不会生成OBJ文件，只会告诉你应该使用什么颜色值！


In [19]:
# 步骤1: 先调整颜色（已经完成）
# 输出: {"base_color": [0.9, 0.7, 0.7]} - 这只是颜色配置，不是OBJ文件

# 步骤2: 使用调整后的颜色生成OBJ文件
!curl -X POST "http://localhost:8000/convert-to-obj" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "local_path=static/processed/snowflake_silhouette.png" \
     -d "depth_div_width=0.2" \
     -d "aspect_ratio=1.0" \
     -d "base_color_r=0.9" \
     -d "base_color_g=0.7" \
     -d "base_color_b=0.7" \
     -d "use_depth_coloring=false"

{"success":true,"obj_url":"/static/models/snowflake_silhouette_3d.obj","obj_path":"static/models/snowflake_silhouette_3d.obj","mtl_url":"/static/models/snowflake_silhouette_3d.mtl","mtl_path":"static/models/snowflake_silhouette_3d.mtl"}

### 完整示例：从灰色调整到粉红色

**场景**: 我有一个灰色的OBJ模型，想改成粉红色

**完整流程**:
```
1. 调用 /adjust-colors，告诉它"make it more pink"
   → 获得新颜色: [1.0, 0.75, 0.80]

2. 使用新颜色调用 /convert-to-obj
   → 生成新的OBJ文件（粉红色）

3. 下载 /download-model/{filename}
   → 获得最终的粉红色3D模型
```


In [20]:
# 完整示例: 灰色 → 粉红色

# Step 1: 询问AI应该用什么粉红色
!curl -X POST "http://localhost:8000/adjust-colors" \
     -d "user_instruction=make it more pink" \
     -d 'current_config={"mode":"single","base_color":[0.8,0.8,0.8]}'

# 假设返回: {"base_color": [1.0, 0.75, 0.80]}

# Step 2: 用返回的颜色生成OBJ（需要手动复制上面的RGB值）
# 注意：这一步需要你手动执行！

{"success":true,"updated_config":{"mode":"single","base_color":[0.9,0.7,0.8],"explanation":"Increased red channel to 0.9 and adjusted green and blue to 0.7 and 0.8 respectively to create a more pink tone"},"explanation":"Increased red channel to 0.9 and adjusted green and blue to 0.7 and 0.8 respectively to create a more pink tone","original_instruction":"make it more pink"}

In [21]:
# Step 2 (续): 使用上面返回的颜色值生成实际的粉红色OBJ文件
!curl -X POST "http://localhost:8000/convert-to-obj" \
     -d "local_path=static/processed/snowflake_silhouette.png" \
     -d "depth_div_width=0.2" \
     -d "base_color_r=1.0" \
     -d "base_color_g=0.75" \
     -d "base_color_b=0.80" \
     -d "use_depth_coloring=false"

# 这一步会返回: {"obj_url": "/static/models/snowflake_silhouette_3d.obj", ...}
# OBJ文件会保存在: static/models/snowflake_silhouette_3d.obj

{"success":true,"obj_url":"/static/models/snowflake_silhouette_3d.obj","obj_path":"static/models/snowflake_silhouette_3d.obj","mtl_url":"/static/models/snowflake_silhouette_3d.mtl","mtl_path":"static/models/snowflake_silhouette_3d.mtl"}

In [2]:
# Adjust single color: "add more pink tone"
!curl -X POST "http://localhost:8000/adjust-colors" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "user_instruction=add more pink tone" \
     -d 'current_config={"mode":"single","base_color":[0.9,0.7,0.7]}'

{"detail":"Color adjustment failed: 'ImageService' object has no attribute 'adjust_model_colors'"}

In [5]:
# Adjust single color: "not blue enough, make it more saturated"
!curl -X POST "http://localhost:8000/adjust-colors" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "user_instruction=not blue enough, make it more saturated" \
     -d 'current_config={"mode":"single","base_color":[0.5,0.5,0.7]}'

{"detail":"Color adjustment failed: 'ImageService' object has no attribute 'adjust_model_colors'"}

In [6]:
# Adjust regional colors: "center should be more pink, edges need to be darker"
!curl -X POST "http://localhost:8000/adjust-colors" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "user_instruction=center should be more pink, edges need to be darker" \
     -d 'current_config={"mode":"regional","color_config":[{"area":"center","color":[1.0,0.5,0.0]},{"area":"edges","color":[0.0,0.5,1.0]}]}'

{"detail":"Color adjustment failed: 'ImageService' object has no attribute 'adjust_model_colors'"}

In [7]:
# Adjust regional colors: "整体偏橙色一点" (Make it a bit more orange overall - in Chinese)
!curl -X POST "http://localhost:8000/adjust-colors" \
     -H "accept: application/json" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "user_instruction=整体偏橙色一点" \
     -d 'current_config={"mode":"regional","color_config":[{"area":"center","color":[1.0,0.5,0.0]},{"area":"top","color":[0.0,1.0,0.0]},{"area":"bottom","color":[1.0,1.0,0.0]},{"area":"edges","color":[0.5,0.0,1.0]}]}'

{"detail":"Color adjustment failed: 'ImageService' object has no attribute 'adjust_model_colors'"}